<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Function Calling with Assistants - Exercise Solution

**OVERALL TASK: Given the function shown below, create an assistant that can create arguments for the function, send them back via a message, and then you will run the function to get a user input and send that information back to the assistant, and the grab the final answer check response from the assistant.**


- Executed in colab

----

## **TASK: Review the function below and make sure you understand it.**

In [1]:
def president_home_state(president,state_options):
    '''
    INPUTS:
        president str = The string name of a US President
        state_options list[str] = A list of potential states that the President was born in, with one of them being correct!

    OUTPUTS:
        response str = The response the user chose as the correct birthplace state.

    '''
    print("Hello! Let's test your knowledge of the home states of US Presidents!")
    print(f"In what state was this president born: {president}\n")

    for num,option in enumerate(state_options):
        print('\n')
        print(f"Definition #{num} is: {option}")

    print('\n')
    num_choice = input("What is your choice? (Return the single number option)")

    return state_options[int(num_choice)]


In [3]:
president_home_state("George Washington", ["Virginia", "New York", "Massachusetts"])


Hello! Let's test your knowledge of the home states of US Presidents!
In what state was this president born: George Washington



Definition #0 is: Virginia


Definition #1 is: New York


Definition #2 is: Massachusetts


What is your choice? (Return the single number option)1


'New York'

## **TASK: Convert the Python function above into a JSON Description. Here is a general guide to help you:**

     tool = {'type':'function',
                    'function':{
                        'name': 'my_function_name',
                        'parameters':{
                            "type":"object",
                              "properties":{
                                  "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                                  "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                              },
                            'required' : ['parameter_one','parameter_two']
                        }

            }
        }

In [4]:
function_json = {'type':'function',
            'function':{
                'name': 'president_home_state',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "president": {'type':'string','description':"The name of a random US President"},
                          "state_options":{'type':'array',
                                        'items':{'type':'string'},
                                        'description':"A Python list of strings of US States, where one of the states is the birthplace state of the US President."}
                      },
                    'required' : ["president","state_options"]
                }

    }
}

## **TASK: Create the OpenAI API Client Connection**

In [5]:
from openai import OpenAI
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI()

## **TASK: Create the Assistant with the Function Calling Tool**

In [41]:
assistant = client.beta.assistants.create(
    name='India Presidents Quiz Bot',
  instructions="You help create a quiz where you give a US President and a list of birthplace states, where only one is the correct birthplace state of the president. Later you check if answers returned are correct.",
  model="gpt-3.5-turbo",
  tools=[function_json]
)

## **TASK: Create a new thread.**

In [42]:
thread = client.beta.threads.create()

**TASK: Create a new message, requesting a new US President and State Options List. Then let the model know you will later send back a reply and ask the assistant to check if its correct.**

In [43]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a new quiz question with a Indian President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.",
)

## **TASK: Run the thread with the assistant.**

In [44]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

## **TASK: Confirm that the run.status requires action**

In [45]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(1)
    return run

In [46]:
run = wait_on_run(run,thread)

In [47]:
run.status

'requires_action'

## **TASK: Retrieve the Run result arguments for the tool and pass them into the president_home_state() function**

In [48]:
import json

# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: president_home_state
Function Arguments:


{'president': 'Ram Nath Kovind',
 'state_options': ['Uttar Pradesh', 'Bihar', 'Tamil Nadu']}

In [49]:
response = president_home_state(arguments['president'],arguments['state_options'])

Hello! Let's test your knowledge of the home states of US Presidents!
In what state was this president born: Ram Nath Kovind



Definition #0 is: Uttar Pradesh


Definition #1 is: Bihar


Definition #2 is: Tamil Nadu


What is your choice? (Return the single number option)2


In [50]:
response

'Tamil Nadu'

In [51]:
tool_call.id

'call_0avTtsDEePArRwLOxEFzlJ43'

## **TASK: Return the local function response to the tool in the assistant.**

In [52]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(response),
        }
    ],
)

In [53]:
run = wait_on_run(run, thread)

In [54]:
run.status

'completed'

## **TASK: Retrieve the Messages and see if you got the state correct!**

In [55]:
messages = client.beta.threads.messages.list(thread.id)

In [56]:
messages
#No ans displayed

SyncCursorPage[Message](data=[Message(id='msg_G146gEgzq6XMarWwD1Lzrbr7', assistant_id='asst_HlOsinDeafJ2D1zxWVC6ewqJ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Where was the Indian President Ram Nath Kovind born?\n1. Uttar Pradesh\n2. Bihar\n3. Tamil Nadu\n\nPlease select the correct state from the options given above.'), type='text')], created_at=1736055726, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_gQg27hxKW1wKbiZBYIjNWJMT', status=None, thread_id='thread_yAS0ZJ1rjMNvITQPeaZe1kWS'), Message(id='msg_eKP7nuv4xUt4vlpv4ZhPnSxf', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create a new quiz question with a Indian President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.'), type='text')], created_at=1736055700, incomplete_at=

In [57]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

Where was the Indian President Ram Nath Kovind born?
1. Uttar Pradesh
2. Bihar
3. Tamil Nadu

Please select the correct state from the options given above.


Create a new quiz question with a Indian President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.




# Here ans not displayed in final assistance response. Yet tos ee why

## **TASK: Delete the Assistant**

In [58]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)